## 1. urllib 활용

In [2]:
import requests as r
import urllib.request as ur
import os
import pandas as pd
import time

In [3]:
# 해당 디렉토리가 없으면 만드는 함수
def createFolder(dir_name):
    try:
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
    except OSError:
        print ('Error: Creating directory. ' +  dir_name)

In [7]:
key1 = "ed65304c3c8644c0a2c757e21ab5efca" # Your bing API key

### 1. function to make image-files for one keyword

In [8]:
def make_imgs(item, apikey):
    # 시작 시간 저장
    start = time.time()
    # 에러 로그 저장을 위한 리스트 만들기
    error_log = []
    
    # 폴더 만들고 경로 이동하기
    os.chdir("./bing_img") # your designated directory
    createFolder(item)
    os.chdir(f"./{item}")
    
    url = "https://api.bing.microsoft.com/v7.0/images/search"
    headers = {
              "Ocp-Apim-Subscription-Key" : key1
              }
    
    # you can manage many query parameters. check out the documentation
    # https://docs.microsoft.com/en-us/bing/search-apis/bing-image-search/reference/query-parameters
    
    param = {"q" : item,
          "count" : 10, #choose num of results from 1 to 150
          "imageType" : "Photo",
          "imageContent" : "Face",
          "minHeight" : 256, # set minimum height(pixels)
          "safeSearch" : "strict", # filter adult contents
          "color" : "ColorOnly"
          }
    s = r.Session()
    test = s.get(url,headers=headers, params=param).json()
    
    # 검색결과 링크에서 이미지 저장하기
    for i in range(0,len(test.get("value"))):
        print(item,'-',i)
        print(test.get('value')[i].get('contentUrl'))
        file_name = f"{item}_{i}.{test.get('value')[i].get('encodingFormat')}"
        try:
          ur.urlretrieve(test.get('value')[i].get('contentUrl'),file_name)
        except Exception as e: 
          print(e)
          error_log.append(f"{file_name}: {e}")

          pass
    os.chdir("../..")
    time_taken = time.time()-start
    return time_taken, error_log

In [9]:
make_imgs("Jimmy Fallon", key1)

Jimmy Fallon - 0
http://content.wkyc.com/photo/2014/01/20/1390222629000-Jimmy-Fallon-Getty-Images_2916070_ver1.0.jpg
Jimmy Fallon - 1
https://www.cleveland.com/resizer/dHX6WHaRRtYQ14gXZypjtq2mOEI=/1280x0/smart/advancelocal-adapter-image-uploads.s3.amazonaws.com/image.cleveland.com/home/cleve-media/width2048/img/ent_impact_home/photo/jimmyfallonjpg-2cca1f111b1f77a9.jpg
Jimmy Fallon - 2
https://i0.gmx.ch/image/324/33631324,pd=1/jimmy-fallon.jpg
Jimmy Fallon - 3
https://www1.pictures.zimbio.com/gi/NBC+Universal+68th+Annual+Golden+Globes+After+sGiTShVJAnDx.jpg
Jimmy Fallon - 4
https://townsquare.media/site/252/files/2020/05/jimmy-fallon.jpg?w=1200&h=0&zc=1&s=0&a=t&q=89
Jimmy Fallon - 5
https://townsquare.media/site/623/files/2011/12/jimmy-fallon-200-111111.jpg?w=1200&h=0&zc=1&s=0&a=t&q=89
Jimmy Fallon - 6
https://brobible.com/wp-content/uploads/2020/05/GettyImages-1060576374.jpg
Jimmy Fallon - 7
https://americanupdate.com/wp-content/uploads/2020/05/Jimmy-Fallon.jpg
HTTP Error 403: Forbidde

(13.53794264793396, ['Jimmy Fallon_7.jpeg: HTTP Error 403: Forbidden'])

### 2. function to make image-file lists for several keywords

In [6]:
# 쿼리를 요청할 검색어 리스트 만들거나 불러오기
name_list = ['Barak Obama','Donald Trump' ,'Joe Biden']

In [10]:
def make_img_list(lst, apikey):
  # 시작 시간 저장
  start = time.time()
  # 에러 로그 저장을 위한 리스트 만들기
  error_log = []
  # 주어진 리스트에 있는 모든 이름 검색
  for item in lst:

    # 폴더 만들고 경로 이동하기
    os.chdir("./bing_img") # your designated directory
    createFolder(item)
    os.chdir(f"./{item}")

    url = "https://api.bing.microsoft.com/v7.0/images/search"
    headers = {
              "Ocp-Apim-Subscription-Key" : key1
              }
    param = {"q" : item,
          "count" : 10,
          "imageType" : "Photo",
          "imageContent" : "Face",
          "minHeight" : 256,
          "safeSearch" : "strict",
          "color" : "ColorOnly"
          }
    s = r.Session()
    test = s.get(url,headers=headers, params=param).json()
    # 검색결과 링크에서 이미지 저장하기

    for i in range(0,len(test.get("value"))):
        print(item,'-',i)
        print(test.get('value')[i].get('contentUrl'))
        file_name = f"{item}_{i}.{test.get('value')[i].get('encodingFormat')}"
        try:
          ur.urlretrieve(test.get('value')[i].get('contentUrl'),file_name)
        except Exception as e: 
          print(e)
          error_log.append(f"{file_name}: {e}")

          pass
    os.chdir("../..")
  time_taken = time.time()-start
  return time_taken, error_log

In [11]:
make_img_list(name_list, key1)

Barak Obama - 0
https://issuepedia.org/wikiup/thumb/f/f8/Officialportrait.jpg/1200px-Officialportrait.jpg
Barak Obama - 1
https://images-na.ssl-images-amazon.com/images/I/B1iW0a3boMS.jpg
Barak Obama - 2
https://i0.web.de/image/374/33558374,pd=1/barack-obama.jpg
Barak Obama - 3
http://fotomods.ru/upload/img1349524929.jpg
HTTP Error 403: Forbidden
Barak Obama - 4
http://whatwillmatter.com/wp-content/uploads/2012/10/Barak-Obama.jpg
Barak Obama - 5
https://www.espreso.co.rs/data/images/2020/07/17/18/790599_barak-obama_ls.jpg
Barak Obama - 6
https://media.vanityfair.com/photos/5eb6e24a77a607fc7f7f1572/master/w_2560%2Cc_limit/Barack-Obama.jpg
Barak Obama - 7
http://abovethelaw.com/wp-content/uploads/2015/04/barack-obama.jpg
HTTP Error 403: Forbidden
Barak Obama - 8
https://i0.wp.com/breakingwide.com/wp-content/uploads/2019/11/PhotoGrid_1574718654326.jpg?fit=1820%2C1024&ssl=1
Barak Obama - 9
https://www.famousbirthsdeaths.com/wp-content/uploads/2015/12/is-barack-obama-dead-or-alive.jpg
HTTP E